# Objectives {.unnumbered}

By the end of this lab, you will:
1. Load and analyze the **Lightcast dataset** in **Spark DataFrame**.
2. Create **five easy and three medium-complexity visualizations** using **Plotly**.
3. Explore **salary distributions, employment trends, and job postings**.
4. Analyze **skills in relation to NAICS/SOC/ONET codes and salaries**.
5. Customize **colors, fonts, and styles** in all visualizations (**default themes result in a 2.5-point deduction**).
6. Follow **best practices for reporting on data communication**.

# Step 1: Load the Dataset {.unnumbered}


In [1]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.renderers.default = "vscode"
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("./data/lightcast_job_postings.csv")
df.createOrReplaceTempView("jobs")


# Show Schema and Sample Data
df.printSchema()
df.show(5)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/25 03:14:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/25 03:15:00 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


root
 |-- ID: string (nullable = true)
 |-- LAST_UPDATED_DATE: string (nullable = true)
 |-- LAST_UPDATED_TIMESTAMP: timestamp (nullable = true)
 |-- DUPLICATES: integer (nullable = true)
 |-- POSTED: string (nullable = true)
 |-- EXPIRED: string (nullable = true)
 |-- DURATION: integer (nullable = true)
 |-- SOURCE_TYPES: string (nullable = true)
 |-- SOURCES: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- ACTIVE_URLS: string (nullable = true)
 |-- ACTIVE_SOURCES_INFO: string (nullable = true)
 |-- TITLE_RAW: string (nullable = true)
 |-- BODY: string (nullable = true)
 |-- MODELED_EXPIRED: string (nullable = true)
 |-- MODELED_DURATION: integer (nullable = true)
 |-- COMPANY: integer (nullable = true)
 |-- COMPANY_NAME: string (nullable = true)
 |-- COMPANY_RAW: string (nullable = true)
 |-- COMPANY_IS_STAFFING: boolean (nullable = true)
 |-- EDUCATION_LEVELS: string (nullable = true)
 |-- EDUCATION_LEVELS_NAME: string (nullable = true)
 |-- MIN_EDULEVELS: integer (

+--------------------+-----------------+----------------------+----------+--------+---------+--------+--------------------+--------------------+--------------------+-----------+-------------------+--------------------+--------------------+---------------+----------------+--------+--------------------+-----------+-------------------+----------------+---------------------+-------------+-------------------+-------------+------------------+---------------+--------------------+--------------------+--------------------+-------------+------+-----------+----------------+-------------------+---------+-----------+--------------------+--------------------+-------------+------+--------------+-----+--------------------+-----+----------+---------------+--------------------+---------------+--------------------+------------+--------------------+------------+--------------------+------+--------------------+------+--------------------+------+--------------------+------+--------------------+------+------

In [9]:
#template for the graphs to follow
pio.templates["temp"] = go.layout.Template(
    # LAYOUT
    layout = {
        # Fonts
        # Note - 'family' must be a single string, NOT a list or dict!
        'title':
            {'font': {'family': 'Cambria',
                      'size':30,
                      'color': '#333'}
            },
        'font': {'family': 'Cambria',
                      'size':16,
                      'color': '#333'},
        # Colorways
        'colorway': ['#636EFA', '#a4abab'],
        # Keep adding others as needed below
        'hovermode': 'x unified'
    },
    # DATA
    
)

# Salary Distribution by Employment Type
- Identify salary trends across different employment types.
- **Filter the dataset**
  - Remove records where **salary is missing or zero**.
- **Aggregate Data**
  - Group by **employment type** and compute salary distribution.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `EMPLOYMENT_TYPE_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize **colors, fonts, and styles** to avoid a **2.5-point deduction**.
- **Explanation:** Write two sentences about what the graph reveals.
  - This graph shows that there's a higher and wider range of salaries for full time employees than for the part time employees. It is interesting to note that there are significant outliers for the Part-Time/Full-Time and Full-Time employees compared to Part-Time employees.


In [10]:
# Your Code for 1st question here
pdf = df.select("EMPLOYMENT_TYPE_NAME", "SALARY").dropna().filter("SALARY > 0").toPandas()
fig1 = px.box(pdf, x="EMPLOYMENT_TYPE_NAME",  y="SALARY", title="Salary Distribution by Employment Type", template="temp")
fig1.update_layout(title_font_size=20)
fig1.show()

# Salary Distribution by Industry
- Compare salary variations across industries.
- **Filter the dataset**
  - Keep records where **salary is greater than zero**.
- **Aggregate Data**
  - Group by **NAICS industry codes**.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `NAICS2_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize colors, fonts, and styles.
- **Explanation:** Write two sentences about what the graph reveals.
  - This graph shows the salary ranges that different job fields have specifically the different between jobs in humanities and STEM. STEM typically has a wider and higher range in salary whereas the arts are very low with healthcare and waste management having significant outliers which is very interesting to note.

In [11]:
pdf2 = df.select("NAICS2_NAME", "SALARY_FROM").filter("SALARY_FROM > 0").toPandas()
fig2 = px.box(pdf2, x="NAICS2_NAME", y="SALARY_FROM", title="Salary Distribution by Industry", template="temp") 
fig2.update_layout(title_font_size=20, xaxis=dict(tickfont=dict(size=10)))  
fig2.show()

# Job Posting Trends Over Time
- Analyze how job postings fluctuate over time.
- **Aggregate Data**
  - Count job postings per **posted date (`POSTED`)**.
- **Visualize results**
  - Create a **line chart** where:
    - **X-axis** = `POSTED`
    - **Y-axis** = `Number of Job Postings`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.
  - This graph reveals the trend of when jobs are posted on LinkedIn over the course of 4 months. It is interesting to note that the spikes seem to correlated with when colleges end and start classes again in the spring and fall while the lull is in the midst of summer.

In [12]:
# Your code for 3rd question here
posted_count = spark.sql("""
    SELECT POSTED, COUNT(*) AS posted_count
    FROM jobs
    GROUP BY POSTED
    ORDER BY POSTED
""")

pdf3 = posted_count.toPandas()
fig3 = px.line(pdf3, x="POSTED", y="posted_count", title="Job Posting Trends Over Time", markers=True,template="temp")
fig3.update_layout(font_family="Cambria", title_font_size=20,xaxis=dict(tickfont=dict(size=10)))
fig3.show()

# Top 10 Job Titles by Count
- Identify the most frequently posted job titles.
- **Aggregate Data**
  - Count the occurrences of each **job title (`TITLE_NAME`)**.
  - Select the **top 10 most frequent titles**.
- **Visualize results**
  - Create a **bar chart** where:
    - **X-axis** = `TITLE_NAME`
    - **Y-axis** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.
  - This graph shows the top 10 most frequently posted job titles through a bar graph. It should be noted that the graph skews right and most of the job listings relate to Data or Analytics. 

In [13]:
# Your code for 4th question here
job_count = spark.sql("""
    SELECT TITLE_NAME, COUNT(*) AS job_count
    FROM jobs
    GROUP BY TITLE_NAME
    ORDER BY job_count DESC
    LIMIT 10
""")

pdf4 = job_count.toPandas()
fig4 = px.bar(pdf4,  x="TITLE_NAME", y="job_count", title="Top 10 Job Titles by Count", template="temp") 
fig4.update_layout(font_family="Cambria", title_font_size=20,xaxis=dict(tickfont=dict(size=10),  tickangle=45))
fig4.show()

# Remote vs On-Site Job Postings
- Compare the proportion of remote and on-site job postings.
- **Aggregate Data**
  - Count job postings by **remote type (`REMOTE_TYPE_NAME`)**.
- **Visualize results**
  - Create a **pie chart** where:
    - **Labels** = `REMOTE_TYPE_NAME`
    - **Values** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.
  - This graph shows the pie chart of the different types of location types that a job posting has listed (i.e. remote vs in person). It should be noted that while the majority of the listings are unclassified the majority of the ones that are labeled are remote which is surprising considering most jobs are transitioning to a hybrid/fully in-person model.

In [14]:
# Your code for 5th question here
remotetype = spark.sql("""
    SELECT REMOTE_TYPE_NAME, COUNT(*) AS remotetype
    FROM jobs
    GROUP BY REMOTE_TYPE_NAME
    ORDER BY remotetype DESC
""")

pdf5 = remotetype.toPandas()
fig5 = px.pie(pdf5, names="REMOTE_TYPE_NAME", values="remotetype", title="Remote vs On-Site Job Postings", 
              color_discrete_sequence=px.colors.sequential.Blues, template="temp")
fig5.update_layout(font_family="Cambria", title_font_size=20)
fig5.show()


# Skill Demand Analysis by Industry (Stacked Bar Chart)
- Identify which skills are most in demand in various industries.
- **Aggregate Data**
  - Extract **skills** from job postings.
  - Count occurrences of skills grouped by **NAICS industry codes**.
- **Visualize results**
  - Create a **stacked bar chart** where:
    - **X-axis** = `Industry`
    - **Y-axis** = `Skill Count`
    - **Color** = `Skill`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.
  - This graphs shows the skills demanded by each industry

In [ ]:
# Your code for 6th question here
skilldemand = spark.sql("""
    SELECT 
        NAICS4_NAME AS Industry,
        SKILLS_NAME AS Skill,
        COUNT(*) AS SkillCount
    FROM jobs
    GROUP BY NAICS4_NAME, SKILLS_NAME
""")

pdf6 = skilldemand.toPandas()
fig6 = px.bar(pdf6, x="Industry", y="SkillCount", color="Skill", title="Skill Demand", template="temp")
fig6.update_layout(font_family="Cambria",title_font_size=20,xaxis_tickangle=45,xaxis_tickfont=dict(size=10),yaxis_tickfont=dict(size=10))
fig6.show()



# Salary Analysis by ONET Occupation Type (Bubble Chart)
- Analyze how salaries differ across ONET occupation types.
- **Aggregate Data**
  - Compute **median salary** for each occupation in the **ONET taxonomy**.
- **Visualize results**
  - Create a **bubble chart** where:
    - **X-axis** = `ONET_NAME`
    - **Y-axis** = `Median Salary`
    - **Size** = Number of job postings
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.
  - This looks wrong, but it shows the median salary for ONET names. In this case it only pulled one and it shows that the median salary for it is $116.3K for the singular name that it pulled.

In [15]:
# Your code for 7th question here
mediansalary = spark.sql("""
    SELECT ONET, 
           PERCENTILE(SALARY, 0.5) AS mediansalary
    FROM jobs
    WHERE SALARY > 0 
    GROUP BY ONET
    ORDER BY mediansalary DESC
""")

pdf7 = mediansalary.toPandas()
fig7 = px.scatter(pdf7, x="ONET", y="mediansalary", title="Salary Analysis by ONET Name",  template="temp")
fig7.update_layout(font_family="Cambria", title_font_size=20, xaxis_tickangle=45,xaxis_tickfont=dict(size=10),yaxis_tickfont=dict(size=10))
fig7.show()

# Career Pathway Trends (Sankey Diagram)
- Visualize job transitions between different occupation levels.
- **Aggregate Data**
  - Identify career transitions between **SOC job classifications**.
- **Visualize results**
  - Create a **Sankey diagram** where:
    - **Source** = `SOC_2021_2_NAME`
    - **Target** = `SOC_2021_3_NAME`
    - **Value** = Number of transitions
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.
  -This reveals two SOC pathways: Computer and Mathematical Occupation and Mathematical Science Occupation. This shows where the two compare though it seems like there isn't much of a difference between the two.

In [8]:
# Your code for 8th question here
soc = spark.sql("""
    SELECT 
        SOC_2021_2_NAME AS Source, 
        SOC_2021_3_NAME AS Target, 
        COUNT(*) AS value
    FROM jobs
    WHERE SOC_2021_2_NAME IS NOT NULL AND SOC_2021_3_NAME IS NOT NULL
    GROUP BY SOC_2021_2_NAME, SOC_2021_3_NAME
    ORDER BY value DESC
""")

pdf8 = soc.toPandas()
labels = pd.unique(pdf8[["Source","Target"]].values.ravel())
loop = {label: i for i, label in enumerate(labels)}

pdf8["source_idx"] = pdf8["Source"].map(loop)
pdf8["target_idx"] = pdf8["Target"].map(loop)

fig8 = go.Figure(data=[go.Sankey(
    node=dict(pad=15,thickness=20,line=dict(color="black", width=0.5),label=labels),
    link=dict(source=pdf8["source_idx"],target=pdf8["target_idx"],value=pdf8["value"])
)])

fig8.update_layout(title_text="Career Pathway Trends: SOC 2-Digit to 3-Digit Transitions",
    font=dict(family="Cambria", size=14, color="#333"), template="temp")

fig8.show()

